# **CEMiTool Workflow** Jupyter Implementation

This notebook serves as an alternative to `workflow.py`. It performs the same function, but provided better explanations with Jupyter Notebook cells. This makes this file a little easier to follow the script and its flow, while the original script is best for quick runs.

---


## Step 1 - Initial Work

**Initial Imports**: Import needed libraries, and set `dir` to file's location.

In [7]:
import os
from pathlib import Path
dir = Path(os.path.abspath('')) # sets dir variable

# imports
import pandas as pd
import numpy as np
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

# activates conversion between pandas and R dataframes
pandas2ri.activate()


**Loading main gene/sample dataframe**: this file is loaded into the variable `df`

*Note*: Rather than at the beginning of the script, each settings option in this version is written in a block directly on top of its associated block.

In [8]:
# SETTINGS

# Path to gene expression file and its separator
# file must:
#   - have samples as columns and genes as rows
#   - fill first row with sample IDs
#   - fill first column with gene IDs
expression_fpath = dir / "data/expression.txt"
expression_separator = "\t"

In [9]:
# -- Loading DataFrame (df) --
df = pd.read_csv(expression_fpath, sep=expression_separator)
df.columns = df.columns.str.strip()

---
## Step 2 - Preprocessing (Filtering)

**Common Preprocessing Functions**: These functions are used by all preprocessing methods, and must be defined first.

In [10]:
# returns the number of columns and rows in a df
def get_df_size(df):
    return len(df.columns), len(df)


# prints the change in columns and rows between two states of a df
def print_df_changes(step_name, initial_size, final_size):
    print(f"{step_name}:")
    print(
        f"\tColumns changed from {initial_size[0]} to {final_size[0]}, rows changed from {initial_size[1]} to {final_size[1]}"
    )

**Sample Information Filter**: Reads a sample information file, and filters all samples that don't contain specified information.

In [11]:
# SETTINGS

# Settings for filtering by sample information
#
# Samples must be listed in the same order as in the expression file,
# and the file must contain a named row: a so-called 'match field'
# that contains some string that must be matched. In this case,
# that string is "areaX". All samples not containing the string in the
# match field will be dropped.
#
# To disable, set sample information fpath to None
sample_information_fpath = dir / "data/sample info.txt"
sample_information_separator = "\t"

match_field_rowname = "Sample_number"
match_value = "areaX"

In [12]:
if sample_information_fpath is not None:
        # load file
        labels = pd.read_csv(sample_information_fpath, sep=sample_information_separator)
        labels.set_index(labels.columns[0], inplace=True)
        labels = labels.transpose()

        initial_size = get_df_size(df)  # stores the size of df before sample filtering

        # loop for matches and drop columns
        for col_name in df:
            col_type = labels.loc[labels[match_field_rowname] == col_name].index[0]

            if match_value not in col_type:
                df = df.drop(col_name, axis=1)

        print_df_changes(
            "Sample information filtering", initial_size, get_df_size(df)
        )  # prints change in df

Sample information filtering:
	Columns changed from 54 to 27, rows changed from 20104 to 20104


**NA \& Sum Filtering**: Replaces NA and NaN values with 0, and removes samples with irregular sums dependings on order of magnitude.

In [13]:
# SETTINGS

# Settings for filtering NA values and by sample sum thresholds.
#
# Replaces NA values with 0, and removes samples with a sum outside a certain
# order of magnitude of the median of all other sample sums. The
# order_of_magnitude_threshold setting sets the order needed for a sample to be removed.
#
# To disable, set order_of_magnitude_threshold to None
order_of_magnitude_threshold = 2

In [14]:
if order_of_magnitude_threshold is not None:
        initial_size = get_df_size(df)  # stores the size of df before sample filtering

        df.fillna(0)  # replace all NA and NaN values with 0

        # compute sums and medians
        sums = {col_name: np.sum(col_data) for col_name, col_data in df.items()}
        median = np.median(list(sums.values()))

        # loop for sums outside order of magnitude threshold
        for col_name, _ in df.items():
            sum = sums[col_name]
            ratio = sum / median

            if ratio > (10**order_of_magnitude_threshold) or ratio < (
                10**-order_of_magnitude_threshold
            ):
                df = df.drop(col_name, axis=1)

        print_df_changes(
            "NA/Sum filtering", initial_size, get_df_size(df)
        )  # prints change in df

NA/Sum filtering:
	Columns changed from 27 to 26, rows changed from 20104 to 20104


**Standard Deviation Filtering**: Removes genes with values outside a threshold of standard deviations from its sample distribution.

In [15]:
# SETTINGS

# Settings for filtering by gene standard deviation thresholds
#
# All genes with values greater than two standard deviations outside the other data in the sample
# causes the entire gene to be removed. This controls for abnormal and poor data
#
# To disable, set num_deviations_threshold to None
num_deviations_threshold = 2

In [16]:
if num_deviations_threshold is not None:
        initial_size = get_df_size(df)  # stores the size of df before sample filtering

        # compute means and standard deviations
        means_and_devs = {
            col_name: (np.mean(col_data), np.std(col_data))
            for col_name, col_data in df.items()
        }

        # loop for values outside standard deviation threshold
        for col_name, col_data in df.items():
            mean, dev = means_and_devs[col_name]

            upper_bound = mean + (dev * num_deviations_threshold)
            lower_bound = mean - (dev * num_deviations_threshold)

            for idx, item in col_data.items():
                if (item < lower_bound) or (item > upper_bound):
                    # mark cells that meet criteria
                    df.at[idx, col_name] = np.nan

        # remove marked rows
        df = df.dropna()  # FIXME: Data hazard when NA filtering is disabled.

        print_df_changes(
            "Standard deviation filtering", initial_size, get_df_size(df)
        )  # prints change in df

Standard deviation filtering:
	Columns changed from 26 to 26, rows changed from 20104 to 16748


*(Experimental)* **GWENA Filtering**: Uses built-in variance filtering from the module analysis package GWENA. Implementation currently has problems.

In [17]:
# SETTINGS

# Settings for experimental GWENA filtering
#
# Uses built-in filtering in the GWENA package. Currently, this filter is likely not being used properly,
# as nothing is every filtered out
gwena_filter_enable = False

In [18]:
if gwena_filter_enable:
        initial_size = get_df_size(df)  # stores the size of df before sample filtering

        limma = importr("limma")
        df_norm = limma.normalizeBetweenArrays(df, method="quantile")
        df[:] = df_norm

        gwena = importr("GWENA")  # imports GWENA package
        df_filter = gwena.filter_low_var(df)  # calls GWENA filtering function
        df = pd.DataFrame(df_filter).transpose()  # converts result back to pandas DataFrame

        print_df_changes(
            "GWENA filtering", initial_size, get_df_size(df)
        )  # prints change in df

---
## Step 3 - Running in CEMiTool

**Generate Modules**: A `cem` object is created as per the CEMiTool docs, which contains computed module information.

*(Note)*: This is where any problems with CEMiTool would likely occur, as it does the bulk of the module generation work.

In [19]:
cemitool = importr("CEMiTool")  # import CEMiTool
doParallel = importr("doParallel")  # import doParallel

doParallel.registerDoParallel(cores=8)  # configure doParallel
print("Cemitool and parallel initialized")

cem = cemitool.cemitool(df, plot=True, verbose=True)  # run CEMiTool

Cemitool and parallel initialized


R[write to console]: Plotting diagnostic plots ...

R[write to console]: ...Plotting mean and variance scatterplot ...

R[write to console]: ...Plotting expression histogram ...

R[write to console]: ...Plotting qq plot ...

R[write to console]: ...Plotting sample tree ...

R[write to console]: Finding modules ...

R[write to console]: Selecting Beta



pickSoftThreshold: will use block size 1293.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 1293 of 1293
   Power SFT.R.sq  slope truncated.R.sq mean.k. median.k. max.k.  Density
1      1    0.059  0.347          0.798 430.000   439.000 659.00 0.333000
2      2    0.240 -0.557          0.907 197.000   190.000 410.00 0.152000
3      3    0.572 -0.935          0.967 104.000    92.100 277.00 0.080300
4      4    0.709 -1.220          0.952  59.800    48.200 198.00 0.046300
5      5    0.786 -1.370          0.970  36.800    27.200 147.00 0.028500
6      6    0.853 -1.460          0.987  23.700    16.500 112.00 0.018300
7      7    0.874 -1.560          0.979  15.900    10.200  86.70 0.012300
8      8    0.856 -1.690          0.963  11.000     6.450  68.70 0.008530
9      9    0.878 -1.700          0.970   7.870     4.280  55.20 0.006090
10    10    0.900 -1.700          0.981   5.760     2.900  45.00 0.004460
11    12    0.917 -1.750       

R[write to console]: Merging modules based on eigengene similarity



 mergeCloseModules: Merging modules whose distance is less than 0.2
   Calculating new MEs...


R[write to console]: Plotting beta x R squared curve ...

R[write to console]: Plotting mean connectivity curve ...

R[write to console]: Generating profile plots ...

R[write to console]: Plotting beta x R squared curve ...

R[write to console]: Plotting mean connectivity curve ...



**Save Reports \& Plots**: The `cem` object is used to generate and save a number of reports and plots. This completes the script.

In [20]:
# SETTING

# Save directory
#
# All plots and reports will be saved to this directory
save_dir = dir / "output"   

In [21]:
cemitool.generate_report(cem, force=True, directory=str(save_dir / "report"))
cemitool.diagnostic_report(cem, force=True, directory=str(save_dir / "diagnostic"))
cemitool.write_files(cem, force=True, directory=str(save_dir / "files"))
cemitool.save_plots(cem, "all", force=True, directory=str(save_dir / "plots"))

R[write to console]: Some plots have not been defined. Please run the appropriate plot functions. Saving available plots.



$M1

$M2

$M3

$M4

$M5

$M6

$M7

$Not.Correlated

$beta_r2_plot

$mean_k_plot



[no name],[13]
[no name],[13]
[no name],[13]
[no name],[13]
[no name],[13]
[no name],[13]
[no name],[13]
